<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">

# AST4310 2023, Project 3

</div>

In [1]:
%matplotlib inline
# Only if you have a high-resolution "retina" display:
%config InlineBackend.figure_format = 'retina'

import numpy

from scipy.special import wofz   # for Voigt function
from scipy.special import gamma  # for ABO broadening
from scipy.special import roots_legendre  # for Gaussian quadrature
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
from scipy.linalg import solve_banded
from scipy.interpolate import CubicSpline
from scipy.integrate import cumtrapz

from numba import njit

import matplotlib.pyplot as plt

from astropy import units
from astropy import constants
from astropy.table import QTable  # To use tables with units
from astropy.modeling.models import BlackBody
from astropy.visualization import quantity_support

quantity_support()
plt.rc('legend', frameon=False)
plt.rc('figure', figsize=(7, 7 / 1.75))  # Larger figure sizes
plt.rc('font', size=12)

i_units = units.Quantity(1, unit="kW m-2 sr-1 nm-1")  # More practical SI units

# Background

In this project you will go beyond simple LTE calculations, and perform numerical computations including scattering (non-LTE for a two-level atom), and compute polarised line profiles in the presence of a magnetic field. You will build on the work you did in Project 3, and explore the validity of commonly-used solutions and approximations.

## 1. Non-LTE Radiative Transfer

### 1.1 The Feautrier method

The Feautrier method is a numerical scheme to solve the radiative transfer equation for the angle-averaged intensity $J_\nu$. By itself, it does not solve the problem in non-LTE, only for a source function that is provided. So, given a source function, the Feautrier method gives the $P_\nu$ quantity, from which $J_\nu$ can be reconstructed:

$$
P_\nu(\tau_\nu, \mu) \equiv \frac{1}{2}\left[I_\nu(\tau_\nu, \mu) + I_\nu(\tau_\nu, -\mu)\right] = \frac{1}{2}\left[I^+ + I^-\right].
$$
From $P_\nu$ we can obtain $J_\nu$:

$$
\begin{aligned}
J_\nu(\tau_i) &= \int_0^{+1} P_\nu(\tau_i, \mu)d\mu \\ 
 &= \sum_{j=1}^m a_j P(\tau_i, \mu_j).
\end{aligned}
$$ 

In matrix form, you can write the step to get $P_\nu$ by solving the equation

\begin{equation}
\mathbf{T}P = S,
\end{equation}

where $\mathbf{T}$ is a tridiagonal matrix. The function `Tmatrix` computes $\mathbf{T}$ for a single frequency and direction on a 1D plane-parallel atmosphere, given an optical depth scale and $\mu$. Below we use the C/Python convention that 0 is the first index of an array, and $n-1$ the last point.

In [5]:
def Tmatrix(τ, μ, format=None):
    tau = τ / μ
    n = len(tau)
    Δτ = numpy.roll(tau, -1) - tau
    Δτm = numpy.roll(Δτ, 1)

    A = 2 / (Δτm * (Δτm + Δτ))
    B = 1 + 2 / (Δτ * Δτm)
    C = 2 / (Δτ * (Δτm + Δτ))

    # Boundary conditions top
    B[0] = 2 / Δτ[0]**2 + 2 / Δτ[0] + 1
    C[0] = 2 / Δτ[0]**2
    # Boundary conditions bottom
    A[n-1] = 2 / (Δτ[n-2] * (Δτ[n-2] + 2))
    B[n-1] = (2 + 2*Δτ[n-2] + Δτ[n-2]**2) / (Δτ[n-2] * (Δτ[n-2] + 2))

    if format == 'banded':  # for use with scipy.linalg.solve_banded
        T = numpy.zeros((3, n))
        T[0, 1:] = -C[:-1]
        T[1] = B
        T[2, :-1] = -A[1:]
        return T
    elif format == 'sparse':  # for use with scipy.sparse.linalg.spsolve
        return diags([-A[1:], B, -C[:-1]], [-1, 0, 1], format='csc')
    else:  # for use with numpy.linalg.solv
        return diags([-A[1:], B, -C[:-1]], [-1, 0, 1]).toarray()

Also useful is the function to compute the angle quadratures:

In [5]:
def quadrature(k=5):
    """
    Returns the nodes and weights form a Gaussian
    quadrature with k points. Rescaled to an interval
    from [0, 1].
    """
    nodes, weights = roots_legendre(k)
    return nodes/2 + 0.5, weights/2

Given a $\mathbf{T}$ matrix, to solve the system for $P$ and then $J$ we can use different approaches. The direct method involves building the $\mathbf{\Lambda}$ explicitly (via matrix inversion of $\mathbf{T}$) and then using $\mathbf{\Lambda}$ to obtain $J$:

$$
\begin{aligned}
\mathbb{1}\vec{P_\mu} &= \mathbf{T}^{-1}_\mu \vec{S}\\
\mathbf{\Lambda} &= \sum_\mu a_\mu \mathbf{T}^{-1}_\mu \\
\vec{J} &= \mathbf{\Lambda} \vec{S}.
\end{aligned}
$$

This can be coded in the function:

In [6]:
def lambda_matrix(tau, k=5):
    n = len(tau)
    Λ = numpy.zeros((n, n))

    for mu, w in zip(*quadrature(k)):
        T = Tmatrix(tau, mu)
        Λ += w * numpy.linalg.inv(T)

    return Λ

An alternative way, which still involves inverting a matrix, but saves computational resources to avoid building a $\mathbf{\Lambda}$ matrix explictly, is to take advantage of the fact that $\mathbf{T}$ is not a dense matrix (but a sparse tridiagonal matrix). Then we can use specialised algorithms that can invert it much faster and implicitly solve the system for $J$. Here is an example.

In [7]:
def lambda_operator_implicit(tau, S):
    J = numpy.zeros_like(S)
    for mu, w in zip(*quadrature()):
        T = Tmatrix(tau, mu, format='banded')
        P = solve_banded((1, 1), T, S)
        J += w * P
    return J

### 1.2 Radiative Transfer with coherent scattering in a 2-level atom

Scattering involves radiation that is non-local, and for the case of coherent scattering (no change in frequency after a collision), and a medium composed of atoms with only two levels, we can express it as a change in the source function. Instead of following only the local conditions (the thermal part via the Planck function $B_\nu$), the source function can now be written as a scattering part plus a thermal part, where the balance is controlled by the photon destruction probability $\varepsilon_\nu$:

$$
\begin{aligned}
S_\nu &= (1-\varepsilon_\nu) J_\nu + \varepsilon_\nu B_\nu \\
S_\nu &= (1-\varepsilon_\nu) \mathbf{\Lambda}_\nu[S_\nu] + \varepsilon_\nu B_\nu \\
\end{aligned}
$$

If we have the $\mathbf{\Lambda}_\nu$ operator in matrix form, and can invert it, we can solve this system for $S_\nu$:
$$
S_\nu = (\mathbb{1}-(1-\varepsilon_\nu)\mathbf{\Lambda}_\nu)^{-1}[\varepsilon_\nu B_\nu].
$$
This direct solution can be coded as follows:

In [8]:
def solve_cs_direct(τ, B, ε):
    n = len(τ)
    Λ = lambda_matrix(τ)
    M = numpy.linalg.inv(numpy.eye(n) - (1 - ε) * Λ)
    S = ε * numpy.dot(M, B)
    J = numpy.dot(Λ, S)
    return S, J

### 1.3 $\Lambda$ iteration

Because of the matrix inversions, and building $\mathbf{\Lambda}$ explicitly, the direct solution is never feasible for realistic computations. Instead, the problem is solved iteratively using a starting guess for $S_\nu$, because often it is possible to compute $J_\nu$ from $S_\nu$ without explicitly building $\mathbf{\Lambda}$ (see example above with the Feautrier method).

The simplest way to solve the system by iteration is the classical $\Lambda$ iteration:

\begin{equation}
S^{(n+1)} = (1-\varepsilon)\mathbf{\Lambda}[S^{(n)}] + \varepsilon B,
\end{equation}

where we need a guess for $S^{(1)}$, often $S^{(1)}=B.$ For the iterative schemes, we can measure the convergence speed by calculating the fractional difference between source functions of successive iterations, e.g.:
\begin{equation}
\delta = \left|\left|\frac{S_\nu^{(n+1)}-S_\nu^{(n)}}{S_\nu^{(n)}}\right| \right|.
\end{equation}

We typically stop the iteration when $\delta\approx 10^{-3}$.

An alternative that can be much more efficient is the approximate (or accelerated) $\Lambda$ iteration (ALI):

\begin{equation}
S^{(n+1)} = (\mathbb{1}-(1-\varepsilon)\mathbf{\Lambda}^*)^{-1}\left[S^\mathrm{FS}-(1-\varepsilon)\mathbf{\Lambda}^*[S^{(n)}]\right],
\end{equation}

where $S^\mathrm{FS}$ is the source function from a formal solution: $S=(1-\varepsilon)J + \varepsilon B$, when you have calculated $J$ implicitly without building $\mathbf{\Lambda}$ (e.g. as above with `lambda_operator_implicit`).

ALI involves building an approximate $\mathbf{\Lambda}^*$ that is easier to construct. The example we will use here is the OAB, or diagonal operator, which is the diagonal of the full $\mathbf{\Lambda}$ operator. But how to obtain the diagonal of a matrix without actually building the matrix in the first place? If you consider the Feautrier method, the full $\mathbf{\Lambda}$ is obtained by the sum over $\mu$ of the inverses of $\mathbf{T}_\mu$ matrices, which are tridiagonal (the inverse of a tridiagonal matrix is NOT tridiagonal). So the diagonal of $\mathbf{\Lambda}$ is the sum of the diagonals of the inverses of tridiagonal matrices. And fortunately there are algorithms to obtain the diagonal from an inverse tridiagonal matrix that do not involve building the full matrix. Here is an example:

In [9]:
@njit
def diag_inverse_tri(B):
    """
    Algorithm of Usmani 1994, Computers Math. Applic. 27, 8, 59-66.
    Does not seem to work for N > 1100. 

    The input is B, an array of shape (3, N), with the diagonal
    values, same syntax as scipy needed by scipy.linalg.solve_banded
    """
    n = B.shape[1]
    a = B[0]
    b = B[1]
    c = B[2]
    θ = numpy.zeros(n+2)
    ϕ = numpy.zeros(n+4)
    diag = numpy.zeros(n)

    θ[0] = 1
    θ[1] = b[0]
    for i in range(2, n+1):
        θ[i] = b[i-1] * θ[i-1] - a[i-1] * c[i-2] * θ[i-2]

    ϕ[n+1] = 1
    ϕ[n] = b[n-1]
    for i in range(n-1, 1, -1):
        ϕ[i] = b[i-1] * ϕ[i+1] - a[i] * c[i-1] * ϕ[i+2]

    for i in range(n):
        diag[i] = θ[i] * ϕ[i+2] / θ[n]

    return diag


## 2. Zeeman Effect and Polarisation

### 2.1 Zeeman Effect

The Zeeman effect is the splitting of magnetic levels due to an external magnetic field. The strength and pattern of the splitting depend on the electronic configuration of the upper and lower levels. For any splitting to occur, the total angular momentum quantum number $J$ needs to be larger than zero. The splitting of a given level occurs for different values of the secondary total angular momentum quantum number $M_j$ (the projections of $J$ over an arbitrary z axis): $M_j = -J, \cdots, J$, so there are $2J + 1$ splits on a given level. Any transitions need to obey the electric dipole selection rule of $\Delta M_j = 0, \pm1$. The amount of splitting in energy is given by:

$$
\Delta E_B = \mu_B B (g^uM_j^u-g^lM_j^l),
$$
where $B\equiv\lVert\vec{B}\rVert$, $\mu_B$ is Bohr's magneton:

$$
\mu_B \equiv \frac{e\hbar}{2m_e},
$$
and $g^u$ and $g^l$ are the Landé factors for the upper and lower levels, for LS coupling given by:

$$
g = 1 + \frac{1}{2}\frac{J(J+1) + S(S+1) - L(L+1)}{J(J+1)}.
$$
For $J=0$, $g=1$.

In wavelength, the Zeeman splitting is given by:

$$
\lambda = \lambda_0 - \Delta\lambda_B (g^uM_j^u-g^lM_j^l),
$$
where $\lambda_0$ is the transition wavelength and 

$$
\Delta\lambda_B = \frac{e}{4\pi m_e c}\lambda^2 B.
$$

When $S=0$ or $g^u=g^l$, we have only three Zeeman components, a regime called normal Zeeman effect. These components correspond to the three scenarios of the selection rule $\Delta M_j = 0, \pm1$, and are called $\pi$ ($\Delta M_j=0$), $\sigma_r$ ($\Delta M_j=+1$), and $\sigma_b$ ($\Delta M_j=-1$). For other situations, we have the so-called anomalous Zeeman effect. For transitions with a complex splitting pattern, the effective Landé factor $\bar{g}$ is a useful concept. It measures the distance between $\lambda_0$ and "centre of gravity" of the $\sigma$ components, and is defined as:

\begin{eqnarray*}
\bar{g} &=& \frac{1}{2}(g_1+g_2) + \frac{1}{4}(g_1 - g_2)d\\
d &=& J_1(J_1+1)-J_2(J_2+1).
\end{eqnarray*}

In the anomalous Zeeman effect, not all components have the same strength. This strength is often called $S_q^{J^l J^u}$, is dependent on $q=\Delta M_j$, and $J$ for the upper and lower levels. The $S_q^{J^l J^u}$ are tabulated in [de la Cruz Rodriguez & van Noort (2017, Table 1, page 116)](https://ui.adsabs.harvard.edu/abs/2017SSRv..210..109D/abstract), and are coded in the function `zeeman_strength()` below. 

Each Zeeman component gives rise to one line profile. These are often called $\phi_-$, $\phi_0$, and $\phi_+$, respectively for $\Delta M_j=-1, 0, 1$, and can be written, for the normal Zeeman effect (assuming no macroscopic velocities):

\begin{eqnarray*}
\phi_0 &=& H\left(a, v\right)\\
\phi_\pm &=& H\left(a, v \mp \bar{g}\frac{\Delta\lambda_B}{\Delta\lambda_D} \right),
\end{eqnarray*}
Where $H(a, v)$ is the Voigt profile, and $\Delta\lambda_D$ the Doppler broadening. In the case of more than three components, one needs to sum over all the allowed transitions (for values of $M_j^u$ and $M_j^l$) and weigh the profiles by the Zeeman strength:

$$
\phi_q = \sum_{u, l} S_q^{J^l J^u} H\left(a, v + \left[g^uM_j^u-g^lM_j^l\right]\frac{\Delta\lambda_B}{\Delta\lambda_D} \right),
$$
where $q\equiv\Delta M_j=-1, 0, 1$.

Here are some of the expressions above coded into functions, for your convenience:



In [2]:
def g_LS(L, S, J):
    """
    Calculates Landé factor from LS coupling.
    """
    if J == 0:
        return 1
    else:
        return 1 + 0.5 * (J * (J + 1) + S * (S + 1) - L * (L + 1)) / (J * (J + 1))


def g_eff(g1, g2, J1, J2):
    """
    Calculates the effective Landé factor.
    """
    d = J1 * (J1 + 1) - J2 * (J2 + 1)
    return 0.5 * (g1 + g2) + 0.25 * (g1 - g2) * d


def zeeman_strength(J_l, J_u, M_l, M_u):
    """
    Calculates strengths of Zeeman components.
    """
    J, M = J_l, M_l
    if M_u - M_l == 1:
        if J_u - J_l == 1:
            strength = (3 * (J + M + 1) * (J + M + 2)) / (2 * (J + 1) * (2 * J + 1) * (2 * J + 3))
        elif J_u - J_l == 0:
            strength = (3 * (J - M) * (J + M + 1)) / (2 * J * (J + 1) * (2 * J + 1))
        elif J_u - J_l == -1:
            strength = (3 * (J - M) * (J - M - 1)) / (2 * J * (2 * J - 1) * (2 * J + 1))
        else:
            raise ValueError('Invalid transition, J_u - J_l != -1, 0, 1')
    elif M_u - M_l == 0:
        if J_u - J_l == 1:
            strength = (3 * (J - M + 1) * (J + M + 1)) / ((J + 1) * (2 * J + 1) * (2 * J + 3))
        elif J_u - J_l == 0:
            strength = 3 * M**2 / (J * (J + 1) * (2 * J + 1))
        elif J_u - J_l == -1:
            strength = (3 * (J - M) * (J + M)) / (J * (2 * J - 1) * (2 * J + 1))
        else:
            raise ValueError('Invalid transition, J_u - J_l != -1, 0, 1')
    elif M_u - M_l == -1:
        if J_u - J_l == 1:
            strength = (3 * (J - M + 1) * (J - M + 2)) / (2 * (J + 1) * (2 * J + 1) * (2 * J + 3))
        elif J_u - J_l == 0:
            strength = (3 * (J + M) * (J - M + 1)) / (2 * J * (J + 1) * (2 * J + 1))
        elif J_u - J_l == -1:
            strength = (3 * (J + M) * (J + M - 1)) / (2 * J * (2 * J - 1) * (2 * J + 1))
        else:
            raise ValueError('Invalid transition, J_u - J_l != -1, 0, 1')
    else:
        raise ValueError('Invalid transition, M_u - M_l != -1, 0, 1')
    return strength

### 2.2 Polarised Radiative Transfer Equation

For the polarised case, the radiative transfer equation (RTE) becomes a vector equation, where we no longer solve just for intensity $I_\nu$, but for the Stokes vector $\vec{I_\nu}$:

$$
\vec{I_\nu} = 
\begin{pmatrix}
I_\nu \\
Q_\nu \\
U_\nu \\
V_\nu 
\end{pmatrix}.
$$

The RTE then becomes:
$$
\frac{\mathrm{d}\vec{I_\nu}}{\mathrm{d}s} = \vec{j_\nu} - \mathbf{A}_\nu\vec{I_\nu},
$$

where $\mathbf{A}_\nu$ is the absorption matrix and $\vec{j_\nu}$ the emissivity vector. In the case where there is no continuum emission of polarised radiation (true in most stars), we can simplify $\mathbf{A}_\nu$:

\begin{eqnarray*}
\mathbf{A}_\nu &=& \alpha_\nu^c \mathbb{1} + \mathbf{A}_\nu^l \\
\mathbf{A}_\nu &=& \alpha_\nu^c \mathbb{1} + \alpha_\nu^l \mathbf{\Phi}_\nu,
\end{eqnarray*}

where $\alpha_\nu^c$ is the continuous extinction coefficient and $\alpha_\nu^l$ is the line extinction coefficient  ***without the line profile part*** $H(a, v)$. The radiative transfer equation can then be further simplified by dividing by $\alpha_\nu^c$ and using $\eta_\nu\equiv\alpha_\nu^l / \alpha_\nu^c$:


$$
\frac{\mathrm{d}}{\mathrm{d}\tau_\nu^c}
\begin{pmatrix}
I_\nu \\
Q_\nu \\
U_\nu \\
V_\nu 
\end{pmatrix} = 
\vec{S_\nu} - (\mathbb{1} + \eta_\nu \mathbf{\Phi}_\nu) 
\begin{pmatrix}
I_\nu \\
Q_\nu \\
U_\nu \\
V_\nu 
\end{pmatrix}.
$$





The line profile information goes into the matrix $\mathbf{\Phi}$, which in general can be written as:

$$
\mathbf{\Phi} = 
\begin{pmatrix}
\phi_I & \phi_Q & \phi_U & \phi_V \\
\phi_Q & \phi_I & \psi_V & -\psi_U \\
\phi_U & -\psi_V & \phi_I & \psi_Q \\
\phi_V & \psi_U & -\psi_Q & \phi_I 
\end{pmatrix}. 
$$

This matrix has 7 independent terms, and can be divided into three parts:


$$
\mathbf{\Phi}  = 
\begin{pmatrix}
\phi_I & 0 & 0 & 0 \\
0 & \phi_I & 0 & 0 \\
0 & 0 & \phi_I & 0 \\
0 & 0 & 0 & \phi_I 
\end{pmatrix} +
%
\begin{pmatrix}
0 & \phi_Q & \phi_U & \phi_V \\
\phi_Q & 0 & 0 & 0 \\
\phi_U & 0 & 0 & 0 \\
\phi_V & 0 & 0 & 0 
\end{pmatrix} +
%
\begin{pmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & \psi_V & -\psi_U \\
0 & -\psi_V & 0 & \psi_Q \\
0 & \psi_U & -\psi_Q & 0 
\end{pmatrix}.
$$

The first term represents absorption of energy, the second term represents dichroism, and the last term represents magneto-optical effects.

Under Zeeman effect, for a magnetic field with inclination angle $\gamma$ and azimuthal angle $\chi$, the $\phi$ profiles are given by:

\begin{eqnarray*}
\phi_I &=& \phi_\Delta\sin^2\gamma + \frac{1}{2}(\phi_+ + \phi_-) \\
\phi_Q &=& \phi_\Delta\sin^2\gamma \cos 2\chi \\
\phi_U &=& \phi_\Delta\sin^2\gamma \sin 2\chi \\
\phi_V &=& \frac{1}{2}(\phi_+ - \phi_-) \cos \gamma \\
\phi_\Delta &=& \frac{1}{2} \left[ \phi_0 + \frac{1}{2}(\phi_+ + \phi_-)\right],
\end{eqnarray*}

where $\phi_-$, $\phi_0$, and $\phi_+$ are the line profiles for the Zeeman components defined in section 2.1. The $\psi$ profiles, for the magneto-optical effects, are defined similarly:

\begin{eqnarray*}
\psi_I &=& \psi_\Delta\sin^2\gamma + \frac{1}{2}(\psi_+ + \psi_-) \\
\psi_Q &=& \psi_\Delta\sin^2\gamma \cos 2\chi \\
\psi_U &=& \psi_\Delta\sin^2\gamma \sin 2\chi \\
\psi_V &=& \frac{1}{2}(\psi_+ - \psi_-) \cos \gamma \\
\psi_\Delta &=& \frac{1}{2} \left[ \psi_0 + \frac{1}{2}(\psi_+ + \psi_-)\right],
\end{eqnarray*}

where the $\psi_-$, $\psi_0$, and $\psi_+$ are defined very similarly to the $\phi$ counterparts. E.g. in the case of normal Zeeman effect:

\begin{eqnarray*}
\psi_0 &=& L(a, v)\\
\psi_\pm &=& L\left(a, v \mp \bar{g}\frac{\Delta\lambda_B}{\Delta\lambda_D} \right).
\end{eqnarray*}

And for anomalous Zeeman effect:

$$
\psi_q = \sum_{u, l} S_q^{J^l J^u} L\left(a, v + \left[g^uM_j^u-g^lM_j^l\right]\frac{\Delta\lambda_B}{\Delta\lambda_D} \right).
$$

The only change is from the Voigt profile $H(a, v)$ to the Dispersion function (sometimes called Faraday profile) $L(a, v)$. The two functions are related, as they are respectively the real and imaginary parts of the [Fadeeva function](https://en.wikipedia.org/wiki/Faddeeva_function):

$$
\mathcal{H}(a, v) = H(a, v) + iL(a, v).
$$

In python, both functions can be computed via `scipy.special.wofz`:

In [2]:
def voigt(gamma, x):
    """
    Calculates the Voigt function.
    """
    z = (x + 1j * gamma)
    return wofz(z).real / numpy.sqrt(numpy.pi)


def faraday(gamma, x):
    """
    Calculates the Faraday dispersion function.
    """
    z = (x + 1j * gamma)
    return wofz(z).imag / numpy.sqrt(numpy.pi)

### 2.3 The Unno-Rachkovsky Solution

A simplified solution for the polarised RTE is the Unno-Rachkovsky solution. In this solution, one uses the simplifying assumption that the source function (in LTE, $B_\nu$) varies linearly with the continuum optical depth:

$$
S_\nu = S_0 + S_1\tau_c.
$$

An atmosphere that fulfils the above is called a *Milne-Eddington* atmosphere. Moreover, to greatly simplify the polarised RTE, this solution assumes that everything that affects the absorption matrix $\mathbf{A}_\nu$ is constant with height:

* Magnetic field vector: $B$, $\gamma$, $\chi$
* Ratio of extinctions: $\eta_\nu$
* Broadening of the line: $a$ and $\Delta\lambda_D$.

When the above are fulfilled, there is an exact solution to the polarised RTE, and it can be written explicitly (dropping the frequency indices) as:

\begin{eqnarray*}
I &=& S_0 + \Delta^{-1}\left[k_I(k_I^2+ f_Q^2+f_U^2+f_V^2)\right]S_1 \\
Q &=& -\Delta^{-1} \left[k_I^2 k_Q + k_I(k_V f_U- k_U f_V) + f_Q\Pi\right] S_1 \\
U &=& -\Delta^{-1} \left[k_I^2 k_U + k_I(k_Q f_V- k_V f_Q) + f_U\Pi\right] S_1 \\
V &=& -\Delta^{-1} \left[k_I^2 k_V + f_V\Pi\right] S_1,
\end{eqnarray*}
where 
\begin{eqnarray*}
\Delta & = & k_I^2(k_I^2 - k_Q^2-k_U^2-k_V^2 + f_Q^2+f_U^2+f_V^2) + \Pi^2 \\
\Pi &=& k_Q f_Q + k_U f_U + k_V f_V,
\end{eqnarray*}

and
\begin{eqnarray*}
k_I  =  1 + \eta_\nu \phi_I, \;\; k_Q&=&\eta_\nu \phi_Q, \;\;   k_U = \eta_\nu \phi_U, \;\;   k_V = \eta_\nu \phi_V\\
  f_Q &=& \eta_\nu \psi_Q, \;\;  f_U = \eta_\nu \psi_U, \;\; f_V = \eta_\nu \psi_V.
\end{eqnarray*}

Below you can find an implementation of the Unno-Rachkovsky solution in python. The arguments of this function are the free parameters for the above: $\gamma$, $\chi$, $\eta_\nu$, $a$, and $\Delta\lambda_B / \Delta\lambda_D$. While this last ratio can just be given as a ratio, for detailed calculations one needs to compute explicitly $\Delta\lambda_B$ for a given value of the magnetic field strength. This function only works for the normal Zeeman effect, but modification for arbitrary Zeeman patterns is just a matter of changing $\phi_{0}$, $\phi_\pm$, $\psi_{0}$, and $\psi_\pm$ to the appropriate expressions summing over components.

In [4]:
def unno_rachkovsky(u, s0=1, s1=5, eta=20, a=0.05, g_eff=1, 
                    delta_ratio=1.5, gamma=numpy.pi/3, chi=0):
    """
    Calculates Stokes vector using Unno-Rachkovsky solution, for a given 
    source function S = s0 + s1 * tau.
    
    Parameters
    ----------
    u : 1-D array
        Dimensionless wavelength in Doppler width units
    s0, s1: scalar (astropy intensity units)
        Constants in the definition of source function.
    eta : scalar
        Ratio of line to continuum extinction, alpha_l / alpha_c.
    a: scalar
        Broadening of profile
    u: 1-D array
        Normalised wavelength scale. 
    g_eff: scalar
        Effective Lande factor.
    delta_ratio: scalar
        Ratio of Zeeman broadening to Doppler broadening.
    gamma: scalar
        Inclination angle of magnetic field
    chi: scalar
        Azimuth angle of magnetic field
    """
    phi_0 = voigt(a, u)
    phi_r = voigt(a, u + g_eff * delta_ratio) 
    phi_b = voigt(a, u - g_eff * delta_ratio) 
    psi_0 = faraday(a, u)
    psi_r = faraday(a, u + g_eff * delta_ratio) 
    psi_b = faraday(a, u - g_eff * delta_ratio)
    
    phi_delta = 0.5 * (phi_0 - 0.5 * (phi_b + phi_r))
    phi_I = phi_delta * numpy.sin(gamma)**2 + 0.5 * (phi_b + phi_r)
    phi_Q = phi_delta * numpy.sin(gamma)**2 * numpy.cos(2 * chi)
    phi_U = phi_delta * numpy.sin(gamma)**2 * numpy.sin(2 * chi)
    phi_V = 0.5 * (phi_b - phi_r) * numpy.cos(gamma)
    
    psi_delta = 0.5 * (psi_0 - 0.5 * (psi_b + psi_r))
    psi_Q = psi_delta * numpy.sin(gamma)**2 * numpy.cos(2 * chi)
    psi_U = psi_delta * numpy.sin(gamma)**2 * numpy.sin(2 * chi)
    psi_V = 0.5 * (psi_b - psi_r) * numpy.cos(gamma)
    
    kI = 1 + eta * phi_I
    kQ = eta * phi_Q
    kU = eta * phi_U
    kV = eta * phi_V
    fQ = eta * psi_Q
    fU = eta * psi_U
    fV = eta * psi_V

    delta = (kI**4 + kI**2 * (fQ**2 + fU**2 + fV**2 - kQ**2 - kU**2 - kV**2) - 
             (kQ * fQ + kU * fU + kV * fV)**2)
    I = s0 + s1 / delta * kI * (kI**2 + fQ**2 + fU**2 + fV**2)
    Q = -s1 / delta * (kI**2 * kQ - kI * (kU * fV - kV * fU) + fQ * (kQ * fQ + kU * fU + kV * fV))
    U = -s1 / delta * (kI**2 * kU - kI * (kV * fQ - kQ * fV) + fU * (kQ * fQ + kU * fU + kV * fV))
    V = -s1 / delta * (kI**2 * kV + fV * (kQ * fQ + kU * fU + kV * fV))
    return I, Q, U, V

### 2.4 The FALC model

In this exercise you will work with a semi-empirical 1D model of the solar atmosphere, the so-called FALC model (named after initials of the authors). The FALC model is given in the `falc.dat` file. We can load the file into a useful `astropy.table.QTable` object with the correct units using the metadata in the file:

In [3]:
def read_table_units(filename):
    """
    Reads a table in a text file, formatted with column names in first row,
    and unit names on second row. Any deviation from this format will fail.
    """
    tmp = numpy.genfromtxt(filename, names=True)
    unit_names = open(filename).readlines()[1][1:].split()
    # Convert to astropy QTable to have units
    data = QTable(tmp)
    # Assign units to quantities in table, use SI units
    for key, unit in zip(data.keys(), unit_names):
        data[key].unit = unit
        data[key] = data[key].si  # We don't want to use deprecated units
    return data

In [4]:
falc = read_table_units("falc.dat")
falc

height,tau_500,colmass,temperature,v_turb,hydrogen_density,proton_density,electron_density,pressure,p_ratio,density
m,,kg / m2,K,m / s,1 / m3,1 / m3,1 / m3,N / m2,,kg / m3
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2218200.0,0.0,6.777e-05,100000.0,11730.0,5574999999999999.0,5574999999999999.0,6664999999999999.0,0.01857,0.952,1.3059999999999998e-11
2216500.0,7.696e-10,6.779e-05,95600.0,11650.0,5837999999999999.0,5836999999999999.0,6946999999999999.0,0.01857,0.95,1.3679999999999998e-11
2214890.0,1.531e-09,6.781e-05,90816.0,11560.0,6150999999999999.0,6149999999999999.0,7283999999999999.0,0.01858,0.948,1.4409999999999997e-11
2212770.0,2.597e-09,6.785e-05,83891.0,11420.0,6667999999999999.0,6666999999999999.0,7833999999999999.0,0.018590000000000002,0.945,1.562e-11
2210640.0,3.754e-09,6.788000000000001e-05,75934.0,11250.0,7380999999999999.0,7377999999999999.0,8575999999999999.0,0.018600000000000002,0.941,1.729e-11
2209570.0,4.384e-09,6.79e-05,71336.0,11140.0,7863999999999999.0,7857999999999999.0,9075999999999998.0,0.018600000000000002,0.938,1.8429999999999998e-11
...,...,...,...,...,...,...,...,...,...,...
-40000.0,2.946,55.67,7590.0,1730.0,1.2799999999999998e+23,4.2489999999999993e+20,4.4799999999999993e+20,15250.0,0.971,0.0003


This gives it a structure very similar to the models you worked on in Project 2. Unlike the other models, this model includes the outer layers of the atmosphere (chromosphere and transition region), although you will not need those for this project.